In [ ]:
!pip install rdkit
!pip install pubchempy
!pip install thermo
!pip install Chempy

In [201]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.rdmolops import GetSSSR
import time # To handle potential API rate limits
import pubchempy as pcp
chem_name = ["nonane","1-heptanol","octanal","dodecane","decanal","1-methoxy-decane","2-methyl-dodecane","tridecane","tetradecane"]
# Sample dataset with mixed chemicals (alkanes and non-alkanes)
# SMILES will now be generated dynamically based on 'Chemical Name'


def names_to_smiles(name_list):
    smiles_list = []
    for name in name_list:
        try:
            # Search for the compound by name and return the first result
            results = pcp.get_compounds(name, 'name')
            if results:
                # Append the canonical isomeric SMILES string
                smiles_list.append(results[0].isomeric_smiles)
            else:
                smiles_list.append(f"Not found: {name}")
        except Exception as e:
            smiles_list.append(f"Error for {name}: {e}")
    return smiles_list

# Example usage with a list of chemical names
chemical_names = chem_name
smiles_results = names_to_smiles(chemical_names)

for name, smiles in zip(chemical_names, smiles_results):
    print(f"{name}: {smiles}")
df = pd.DataFrame({'Chemical Name': chem_name, 'SMILES': smiles_results})
print("Dataset:")
display(df)


/tmp/ipython-input-4031055354.py:19: PubChemPyDeprecationWarning: isomeric_smiles is deprecated: Use smiles instead
  smiles_list.append(results[0].isomeric_smiles)


nonane: CCCCCCCCC
1-heptanol: CCCCCCCO
octanal: CCCCCCCC=O
dodecane: CCCCCCCCCCCC
decanal: CCCCCCCCCC=O
1-methoxy-decane: CCCCCCCCCCOC
2-methyl-dodecane: CCCCCCCCCCC(C)C
tridecane: CCCCCCCCCCCCC
tetradecane: CCCCCCCCCCCCCC
Dataset:


,Chemical Name,SMILES
0,nonane,CCCCCCCCC
1,1-heptanol,CCCCCCCO
2,octanal,CCCCCCCC=O
3,dodecane,CCCCCCCCCCCC
4,decanal,CCCCCCCCCC=O
5,1-methoxy-decane,CCCCCCCCCCOC
6,2-methyl-dodecane,CCCCCCCCCCC(C)C
7,tridecane,CCCCCCCCCCCCC
8,tetradecane,CCCCCCCCCCCCCC


In [202]:
from rdkit.Chem import Descriptors

def calculate_MolarMass(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        return Descriptors.MolWt(mol)
    except Exception as e:
        return None

df['Molar Mass'] = df['SMILES'].apply(calculate_MolarMass)

print("DataFrame with Molecular Weight:")
display(df)

DataFrame with Molecular Weight:


,Chemical Name,SMILES,Molar Mass
0,nonane,CCCCCCCCC,128.259
1,1-heptanol,CCCCCCCO,116.204
2,octanal,CCCCCCCC=O,128.215
3,dodecane,CCCCCCCCCCCC,170.340
4,decanal,CCCCCCCCCC=O,156.269
5,1-methoxy-decane,CCCCCCCCCCOC,172.312
6,2-methyl-dodecane,CCCCCCCCCCC(C)C,184.367
7,tridecane,CCCCCCCCCCCCC,184.367
8,tetradecane,CCCCCCCCCCCCCC,198.394


In [203]:
#"nonane","1-heptanol","octanal","dodecane","decanal","1-methoxy-decane","2-methyl-dodecane","tridecane","tetradecane"
mass_values = [.3278,.004,.012,.1365,.02,.059,.0428,.0648,.0089]
df['Mass (g)'] = mass_values
df['Moles'] = df['Mass (g)'] / df['Molar Mass']

print("DataFrame with Mass and Moles:")
display(df)

DataFrame with Mass and Moles:


,Chemical Name,SMILES,Molar Mass,Mass (g),Moles
0,nonane,CCCCCCCCC,128.259,0.3278,0.002556
1,1-heptanol,CCCCCCCO,116.204,0.0040,0.000034
2,octanal,CCCCCCCC=O,128.215,0.0120,0.000094
3,dodecane,CCCCCCCCCCCC,170.340,0.1365,0.000801
4,decanal,CCCCCCCCCC=O,156.269,0.0200,0.000128
5,1-methoxy-decane,CCCCCCCCCCOC,172.312,0.0590,0.000342
6,2-methyl-dodecane,CCCCCCCCCCC(C)C,184.367,0.0428,0.000232
7,tridecane,CCCCCCCCCCCCC,184.367,0.0648,0.000351
8,tetradecane,CCCCCCCCCCCCCC,198.394,0.0089,0.000045


In [217]:
glucose_name = "glucose"
try:
    # Search for glucose by name
    glucose_results = pcp.get_compounds(glucose_name, 'name')
    if glucose_results:
        glucose_smiles = glucose_results[0].smiles
        print(f"Glucose SMILES: {glucose_smiles}")

        # Calculate molar mass using the existing function
        glucose_molar_mass = calculate_MolarMass(glucose_smiles)
        print(f"Glucose Molar Mass: {glucose_molar_mass}")
    else:
        print(f"Glucose not found in PubChem.")
except Exception as e:
    print(f"Error retrieving glucose information: {e}")

Glucose SMILES: C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O
Glucose Molar Mass: 180.156


In [218]:
from rdkit import Chem

def count_elements(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return {'C': None, 'H': None, 'O': None}

        carbon_count = 0
        hydrogen_count = 0
        oxygen_count = 0

        for atom in mol.GetAtoms():
            symbol = atom.GetSymbol()
            if symbol == 'C':
                carbon_count += 1
            elif symbol == 'H':
                hydrogen_count += 1
            elif symbol == 'O':
                oxygen_count += 1
        return {'C': carbon_count, 'H': hydrogen_count, 'O': oxygen_count}
    except Exception as e:
        print(f"Error counting elements for SMILES {smiles}: {e}")
        return {'C': None, 'H': None, 'O': None}

# Retrieve SMILES for the specified chemicals
chemicals_to_analyze = ['1-heptanol', 'octanal', 'decanal']

element_counts = {}

# Add glucose to the analysis
element_counts['glucose'] = count_elements(glucose_smiles)

for chem_name in chemicals_to_analyze:
    smiles = df[df['Chemical Name'] == chem_name]['SMILES'].iloc[0]
    element_counts[chem_name] = count_elements(smiles)

print("Elemental counts (C, H, O):")
for name, counts in element_counts.items():
    print(f"{name}: {counts}")

Elemental counts (C, H, O):
glucose: {'C': 6, 'H': 0, 'O': 6}
1-heptanol: {'C': 7, 'H': 0, 'O': 1}
octanal: {'C': 8, 'H': 0, 'O': 1}
decanal: {'C': 10, 'H': 0, 'O': 1}


In [219]:
from rdkit import Chem

def count_elements(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return {'C': None, 'H': None, 'O': None}

        mol = Chem.AddHs(mol)

        carbon_count = 0
        hydrogen_count = 0
        oxygen_count = 0

        for atom in mol.GetAtoms():
            symbol = atom.GetSymbol()
            if symbol == 'C':
                carbon_count += 1
            elif symbol == 'H':
                hydrogen_count += 1
            elif symbol == 'O':
                oxygen_count += 1
        return {'C': carbon_count, 'H': hydrogen_count, 'O': oxygen_count}
    except Exception as e:
        print(f"Error counting elements for SMILES {smiles}: {e}")
        return {'C': None, 'H': None, 'O': None}


chemicals_to_analyze = ['1-heptanol', 'octanal', 'decanal']

element_counts = {}


element_counts['glucose'] = count_elements(glucose_smiles)

for chem_name in chemicals_to_analyze:
    smiles = df[df['Chemical Name'] == chem_name]['SMILES'].iloc[0]
    element_counts[chem_name] = count_elements(smiles)

print("Elemental counts (C, H, O):")
for name, counts in element_counts.items():
    print(f"{name}: {counts}")

Elemental counts (C, H, O):
glucose: {'C': 6, 'H': 12, 'O': 6}
1-heptanol: {'C': 7, 'H': 16, 'O': 1}
octanal: {'C': 8, 'H': 16, 'O': 1}
decanal: {'C': 10, 'H': 20, 'O': 1}


In [220]:
data_for_df = []

data_for_df.append({
    'Chemical Name': glucose_name,
    'SMILES': glucose_smiles,
    'Molar Mass': glucose_molar_mass,
    'C': element_counts['glucose']['C'],
    'H': element_counts['glucose']['H'],
    'O': element_counts['glucose']['O']
})

for chem_name in chemicals_to_analyze:
    smiles = df[df['Chemical Name'] == chem_name]['SMILES'].iloc[0]
    molar_mass = df[df['Chemical Name'] == chem_name]['Molar Mass'].iloc[0]

    data_for_df.append({
        'Chemical Name': chem_name,
        'SMILES': smiles,
        'Molar Mass': molar_mass,
        'C': element_counts[chem_name]['C'],
        'H': element_counts[chem_name]['H'],
        'O': element_counts[chem_name]['O']
    })

results_df = pd.DataFrame(data_for_df)

print("Comprehensive DataFrame with Molecular and Elemental Data:")
display(results_df)

Comprehensive DataFrame with Molecular and Elemental Data:


,Chemical Name,SMILES,Molar Mass,C,H,O
0,glucose,C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O,180.156,6,12,6
1,1-heptanol,CCCCCCCO,116.204,7,16,1
2,octanal,CCCCCCCC=O,128.215,8,16,1
3,decanal,CCCCCCCCCC=O,156.269,10,20,1


In [236]:
from chempy import balance_stoichiometry
reactants = {'C10H20O1', 'O2'}
reactants2 = {'C8H16O1' , 'O2'}
reactants3 = {'C7H16O', 'O2'}
products = {'C6H12O6'}
products3 = {'C6H12O6', 'CH4'}


try:
    coeffs_left, coeffs_right = balance_stoichiometry(reactants, products)
    equation1 = " + ".join(f"{coeffs_left[r]}{r}" for r in sorted(reactants)) + " -> " + " + ".join(f"{coeffs_right[p]}{p}" for p in sorted(products))
    print(f"{equation1}")

except Exception as e:
    print(f"Could not balance the equation: {e}")

try:
    coeffs_left, coeffs_right = balance_stoichiometry(reactants2, products)
    equation2 = " + ".join(f"{coeffs_left[r]}{r}" for r in sorted(reactants2)) + " -> " + " + ".join(f"{coeffs_right[p]}{p}" for p in sorted(products))
    print(f"{equation2}")

except Exception as e:
    print(f"Could not balance the equation: {e}")

try:
    coeffs_left, coeffs_right = balance_stoichiometry(reactants3, products3)
    equation3 = " + ".join(f"{coeffs_left[r]}{r}" for r in sorted(reactants3)) + " -> " + " + ".join(f"{coeffs_right[p]}{p}" for p in sorted(products3))
    print(f"{equation3}")

except Exception as e:
    print(f"Could not balance the equation: {e}")


6C10H20O1 + 27O2 -> 10C6H12O6
6C8H16O1 + 21O2 -> 8C6H12O6
2C7H16O + 5O2 -> 2C6H12O6 + 2CH4


In [244]:
from thermo import Chemical

enthalpies_of_formation = {}

compounds_to_analyze_thermo = [
    '1-heptanol',
    'octanal',
    'decanal',
    'oxygen',
    'glucose',
    'methane'
]

# Retrieve ΔHf° for each compound
for common_name in compounds_to_analyze_thermo:
    try:

        compound_obj = Chemical(common_name)

        dhf_value = compound_obj.Hfg
        if dhf_value is not None:
            enthalpies_of_formation[common_name] = dhf_value / 1000 # Convert to kJ/mol
        else:
            enthalpies_of_formation[common_name] = "Not available"
    except Exception as e:
        enthalpies_of_formation[common_name] = f"Error: {e}"

print("Standard Enthalpies of Formation (ΔHf°) at 298.15 K (kJ/mol):")
for compound, dhf in enthalpies_of_formation.items():
    print(f"{compound}: {dhf}")

Standard Enthalpies of Formation (ΔHf°) at 298.15 K (kJ/mol):
1-heptanol: -2895.835796730055
octanal: -2276.6972586973893
decanal: -1764.3083680819527
oxygen: 0.0
glucose: -5745.135823487971
methane: -4646.045556604162


In [245]:
results_df['ΔHf° (kJ/mol)'] = results_df['Chemical Name'].map(enthalpies_of_formation)

print("Comprehensive DataFrame with Molecular, Elemental, and Enthalpy Data:")
display(results_df)

Comprehensive DataFrame with Molecular, Elemental, and Enthalpy Data:


,Chemical Name,SMILES,Molar Mass,C,H,O,ΔHf° (kJ/mol)
0,glucose,C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O,180.156,6,12,6,-5745.135823
1,1-heptanol,CCCCCCCO,116.204,7,16,1,-2895.835797
2,octanal,CCCCCCCC=O,128.215,8,16,1,-2276.697259
3,decanal,CCCCCCCCCC=O,156.269,10,20,1,-1764.308368


In [250]:
def calculate_delta_h_reaction(reactants_dict, products_dict, enthalpy_data):

    delta_h_products = 0.0
    delta_h_reactants = 0.0

    # Calculate enthalpy for products
    for product, coeff in products_dict.items():
        if product in enthalpy_data and enthalpy_data[product] != "Not available":
            delta_h_products += coeff * enthalpy_data[product]
        else:
            print(f"Warning: Enthalpy of formation for product '{product}' not found or not available. Cannot calculate ΔH_reaction.")
            return None

    # Calculate enthalpy for reactants
    for reactant, coeff in reactants_dict.items():
        if reactant in enthalpy_data and enthalpy_data[reactant] != "Not available":
            delta_h_reactants += coeff * enthalpy_data[reactant]
        else:
            print(f"Warning: Enthalpy of formation for reactant '{reactant}' not found or not available. Cannot calculate ΔH_reaction.")
            return None

    # Calculate overall enthalpy of reaction using Hess's Law
    delta_h_reaction = delta_h_products - delta_h_reactants
    return delta_h_reaction

# Reaction 1: 6C10H20O1 + 27O2 -> 10C6H12O6
reactants_1 = {'decanal': 6, 'oxygen': 27}
products_1 = {'glucose': 10}
delta_h_1 = calculate_delta_h_reaction(reactants_1, products_1, enthalpies_of_formation)
if delta_h_1 is not None:
    print(f"\nReaction 1: 6C10H20O1 + 27O2 -> 10C6H12O6")
    print(f"Calculated Delta H for Reaction 1: {delta_h_1:.2f} kJ/mol")

# Reaction 2: 6C8H16O1 + 21O2 -> 8C6H12O6
reactants_2 = {'octanal': 6, 'oxygen': 21}
products_2 = {'glucose': 8}
delta_h_2 = calculate_delta_h_reaction(reactants_2, products_2, enthalpies_of_formation)
if delta_h_2 is not None:
    print(f"\nReaction 2: 6C8H16O1 + 21O2 -> 8C6H12O6")
    print(f"Calculated Delta H for Reaction 2: {delta_h_2:.2f} kJ/mol")

# Reaction 3: 2C7H16O + 5O2 -> 2C6H12O6 + 2CH4
reactants_3 = {'1-heptanol': 2, 'oxygen': 5}
products_3 = {'glucose': 2, 'methane': 2}
delta_h_3 = calculate_delta_h_reaction(reactants_3, products_3, enthalpies_of_formation)
if delta_h_3 is not None:
    print(f"\nReaction 3: 2C7H16O + 5O2 -> 2C6H12O6 + 2CH4")
    print(f"Calculated Delta H for Reaction 3: {delta_h_3:.2f} kJ/mol")



Calculating Delta H for each reaction:

Reaction 1: 6C10H20O1 + 27O2 -> 10C6H12O6
Calculated Delta H for Reaction 1: -46865.51 kJ/mol

Reaction 2: 6C8H16O1 + 21O2 -> 8C6H12O6
Calculated Delta H for Reaction 2: -32300.90 kJ/mol

Reaction 3: 2C7H16O + 5O2 -> 2C6H12O6 + 2CH4
Calculated Delta H for Reaction 3: -14990.69 kJ/mol
